In [1]:
import csv
import random
import numpy as np # linear algebra
import pandas as pd 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from joblib import dump

c:\Users\R5 3600\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
'''
Path define
'''
input_path = ''
output_path = '/kaggle/working'
temp_path = '/kaggle/temp'

In [3]:
def data_extract(data_df):
    data = data_df.iloc[:, :].values
    for i, x in enumerate(data_df.dtypes):
        if(x == 'object'): # [product_code, attribute_0, attribute_1]
            if(len(data[0, i])) == 1: #product_code
                data[:, i] = data[:, i].astype('str').view(np.int32) - ord('A')
            else: #attribute
                temp = data[:, i].astype('str')
                data[:, i] = np.array([int(x[-1]) for x in temp])
                
        elif i != 0 and i != len(data[0]) - 1: # not in column 'ID' or 'failure'
            imr = SimpleImputer(strategy = 'median')
            data[:, i] = imr.fit_transform(data[:, i].reshape(-1, 1)).reshape(len(data[:, i]))
            #scaler = StandardScaler()
            #data[:, i] = scaler.fit_transform(data[:, i].reshape(-1, 1)).reshape(len(data[:, i]))
            
    return data[:, 1:] # get rid of [ID]

In [4]:
data_df = pd.read_csv(input_path +'train.csv')
data = data_extract(data_df)
print(data.shape)
print(data)

x = data[:, :-1].astype('float')
y = data[:,-1].astype('bool') # [failure]

(26570, 25)
[[0 80.1 7 ... 14.684 764.1 0]
 [0 84.89 7 ... 15.631 682.057 0]
 [0 82.43 7 ... 17.946 663.376 0]
 ...
 [4 115.62 7 ... 14.065 750.364 0]
 [4 106.38 7 ... 12.865 730.156 0]
 [4 131.2 7 ... 17.89 602.354 0]]


In [5]:
my_model = LogisticRegression(class_weight = 'balanced', max_iter = 1000) # without normalized, need more than 100 iterations to converge.(default = 100)
penalty = ['l2']
C = [0.000001,0.00001, 0.0001, 0.001, 0.01]

parameters = dict(penalty = penalty, C = C)
grid_search = GridSearchCV(my_model, param_grid= parameters, refit = True)
grid_result = grid_search.fit(x, y)

print(grid_result.best_score_)
print(grid_result.best_params_)

for key, value in grid_result.best_params_.items():
    setattr(my_model, key, value)
my_model.fit(x, y)

dump(my_model, '109550110_LR_model.joblib')

0.5952578095596537
{'C': 1e-05, 'penalty': 'l2'}


['109550110_LR_model.joblib']